# Model Eğitimi

[Son not defterinde](https://www.pybroker.com/en/latest/notebooks/5.%20Writing%20Indicators.html), **PyBroker**'da hisse senedi göstergelerinin nasıl yazılacağını öğrendik. Göstergeler bir ticaret stratejisi geliştirmek için iyi bir başlangıç ​​noktasıdır. Ancak başarılı bir strateji oluşturmak için tahmine dayalı modellemeyi kullanan daha karmaşık bir yaklaşıma ihtiyaç duyulması muhtemeldir.

Neyse ki **PyBroker**'ın ana özelliklerinden biri makine öğrenimi modellerini eğitme ve geriye doğru test etme yeteneğidir. Bu modeller, piyasa hareketleri hakkında daha doğru tahminler yapmak için göstergeleri özellik olarak kullanabilir. Bu modeller eğitildikten sonra, bir stratejinin gerçek ticaret sırasında nasıl performans göstereceğini simüle eden [İleriye Doğru Analiz](https://www.youtube.com/watch?v=WBZ_Vv-iMv4) olarak bilinen popüler bir teknik kullanılarak geriye doğru test edilebilir.

İleriye Doğru Analizi bu not defterinin ilerleyen kısımlarında daha derinlemesine açıklayacağız. Ama önce gerekli bazı ithalatlarla başlayalım!

In [1]:
import numpy as np
import pandas as pd
import pybroker
from numba import njit
from pybroker import Strategy, StrategyConfig, YFinance

[DataSource](https://www.pybroker.com/en/latest/reference/pybroker.data.html#pybroker.data.DataSource) ve [Indicator](https://www.pybroker.com/en) ile olduğu gibi /latest/reference/pybroker.indicator.html#pybroker.indicator.Indicator) verileri, **PyBroker** ayrıca eğitimli modelleri diske önbelleğe alabilir. [pybroker.enable_caches](https://www.pybroker.com/en/latest/reference/pybroker.cache.html#pybroker.cache.enable_caches) numaralı telefonu arayarak üçü için de önbelleğe almayı etkinleştirebilirsiniz:

In [2]:
pybroker.enable_caches('walkforward_strategy')

[Son not defterinde](https://www.pybroker.com/en/latest/notebooks/5.%20Writing%20Indicators.html), yakın eksi hareketli ortalamayı (CMMA) hesaplayan bir gösterge uyguladık. [NumPy](https://www.numpy.org) ve [Numba](https://numba.pydata.org/) kullanarak. İşte yine CMMA göstergesinin kodu:

In [3]:
def cmma(bar_data, lookback):

    @njit  # Numba JIT'i etkinleştirin.
    def vec_cmma(values):
        # Sonuç dizisini başlatın.
        n = len(values)
        out = np.array([np.nan for _ in range(n)])
        
        # Yeniden incelemeden başlayan tüm çubuklar için:
        for i in range(lookback, n):
            # Yeniden incelemenin hareketli ortalamasını hesaplayın.
            ma = 0
            for j in range(i - lookback, i):
                ma += values[j]
            ma /= lookback
            # Hareketli ortalamayı değerden çıkarın.
            out[i] = values[i] - ma
        return out
    
    # Yakın fiyatlar için hesaplayın.
    return vec_cmma(bar_data.close)

cmma_20 = pybroker.indicator('cmma_20', cmma, lookback=20)

## Eğitim ve Arka Test

Daha sonra 20 günlük CMMA'yı kullanarak ertesi günün getirisini tahmin eden bir model oluşturmak istiyoruz. [Basit doğrusal regresyonu](https://en.wikipedia.org/wiki/Simple_linear_regression) kullanmak, denemeye başlamak için iyi bir yaklaşımdır. Aşağıda, [scikit-learn](https://scikit-learn.org/) adresinden bir [LinearRegression](https://scikit-learn.org/stable/modules/generate/sklearn.linear_model.LinearRegression.html) modelini içe aktarıyoruz. stabil/):

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

```LinearRegression``` modelini eğitmek için bir ```train_slr``` işlevi oluşturuyoruz:

In [5]:
def train_slr(symbol, train_data, test_data):
    # Train
    # Önceki gün kapanış fiyatları.
    train_prev_close = train_data['close'].shift(1)
    # Günlük getirileri hesaplayın.
    train_daily_returns = (train_data['close'] - train_prev_close) / train_prev_close
    # Ertesi günün dönüşünü tahmin edin.
    train_data['pred'] = train_daily_returns.shift(-1)
    train_data = train_data.dropna()
    # Ertesi günün getirisini tahmin etmek için LinearRegession modelini eğitin
    # 20 günlük CMMA verildi.
    X_train = train_data[['cmma_20']]
    y_train = train_data[['pred']]
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Test
    test_prev_close = test_data['close'].shift(1)
    test_daily_returns = (test_data['close'] - test_prev_close) / test_prev_close
    test_data['pred'] = test_daily_returns.shift(-1)
    test_data = test_data.dropna()
    X_test = test_data[['cmma_20']]
    y_test = test_data[['pred']]
    # Test verilerinden tahminler yapın.
    y_pred = model.predict(X_test)
    # Uyum iyiliğini yazdırın.
    r2 = r2_score(y_test, np.squeeze(y_pred))
    print(symbol, f'R^2={r2}')
    
    # Giriş verileri olarak kullanılacak eğitilmiş modeli ve sütunları döndürün.
    return model, ['cmma_20']

```train_slr``` işlevi, ```LinearRegression``` modeli için giriş özelliği veya tahmin aracı olarak 20 günlük CMMA'yı kullanır. Fonksiyon daha sonra ```LinearRegression``` modelini o hisse senedi sembolünün eğitim verilerine uyarlar.

Modeli yerleştirdikten sonra işlev, özellikle [R-kare](https://en.wikipedia.org/wiki/Cotained_of_determination) puanını hesaplayarak modelin doğruluğunu değerlendirmek için test verilerini kullanır. R-kare puanı, "LinearRegression" modelinin test verilerine ne kadar iyi uyduğunun bir ölçüsünü sağlar.

```train_slr``` fonksiyonunun son çıktısı, giriş verisi olarak kullanılacak olan ```cmma_20``` sütunuyla birlikte, o hisse senedi sembolü için özel olarak eğitilmiş ```LinearRegression``` modelidir. tahminlerde bulunurken. **PyBroker**, geriye dönük test sırasında hisse senedinin ertesi günkü getirisini tahmin etmek için bu modeli kullanacaktır. Her hisse senedi sembolü için ```train_slr``` işlevi çağrılacak ve eğitilen modeller, her bir hisse senedi için bir sonraki günün getirisini tahmin etmek için kullanılacak.

Modeli eğitme işlevi tanımlandıktan sonra **PyBroker**'a kaydedilmesi gerekir. Bu, [pybroker.model](https://www.pybroker.com/en/latest/reference/pybroker.model.html#pybroker.model.model) işlevi. Bu işlevin argümanları, modelin adı (bu durumda ```'slr'```), modeli eğitecek işlev (```train_slr```) ve kullanılacak göstergelerin listesidir modelin girdileri olarak (bu durumda, ```cmma_20```).

In [6]:
model_slr = pybroker.model('slr', train_slr, indicators=[cmma_20])

Eğitilen modeli kullanan bir ticaret stratejisi oluşturmak için, aşağıdakiler kullanılarak yeni bir [Strategy](https://www.pybroker.com/en/latest/reference/pybroker.strategy.html#pybroker.strategy.Strategy) nesnesi oluşturulur. [YFinance](https://www.pybroker.com/en/latest/reference/pybroker.data.html#pybroker.data.YFinance) veri kaynağı ve arka test döneminin başlangıç ​​ve bitiş tarihlerini belirtme.

In [7]:
config = StrategyConfig(bootstrap_sample_size=100)
strategy = Strategy(YFinance(), '3/1/2017', '3/1/2022', config)
strategy.add_execution(None, ['FROTO.IS', 'DOAS.IS'], models=model_slr)

Daha sonra [add_execution](https://www.pybroker.com/en/latest/reference/pybroker.strategy.html#pybroker.strategy.Strategy.add_execution) yöntemi, [Strategy](https://www) üzerinde çağrılır. Bu durumda, ilk argüman olarak "Yok" değeri iletilir; bu, geri test sırasında hiçbir işlem fonksiyonunun kullanılmayacağı anlamına gelir.

Son adım, üzerinde [backtest](https://www.pybroker.com/en/latest/reference/pybroker.strategy.html#pybroker.strategy.Strategy.backtest) yöntemini çağırarak backtest'i çalıştırmaktır. Modelin arka test verilerinin ilk yarısında eğitilmesi ve ikinci yarısında test edilmesi gerektiğini belirtmek için "0,5" değerinde "train_size""ye sahip "Strateji" nesnesi.

In [8]:
strategy.backtest(train_size=0.5)

Backtesting: 2017-03-01 00:00:00 to 2022-03-01 00:00:00

Loading bar data...
[*********************100%***********************]  2 of 2 completed
Loaded bar data: 0:00:02 

Computing indicators...


  0% (0 of 2) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--
 50% (1 of 2) |#############             | Elapsed Time: 0:00:02 ETA:   0:00:02
100% (2 of 2) |##########################| Elapsed Time: 0:00:02 Time:  0:00:02



Train split: 2017-03-01 00:00:00 to 2019-08-13 00:00:00
DOAS.IS R^2=-0.049404174613011165
FROTO.IS R^2=-0.017928050766510273
Finished training models: 0:00:00 

Finished backtest: 0:00:05


TestResult(start_date=datetime.datetime(2017, 3, 1, 0, 0), end_date=datetime.datetime(2022, 3, 1, 0, 0), portfolio=Empty DataFrame
Columns: []
Index: [], positions=Empty DataFrame
Columns: []
Index: [], orders=Empty DataFrame
Columns: []
Index: [], trades=Empty DataFrame
Columns: []
Index: [], metrics=EvalMetrics(trade_count=0, initial_market_value=0, end_market_value=0, total_pnl=0, unrealized_pnl=0, total_return_pct=0, annual_return_pct=None, total_profit=0, total_loss=0, total_fees=0, max_drawdown=0, max_drawdown_pct=0, win_rate=0, loss_rate=0, winning_trades=0, losing_trades=0, avg_pnl=0, avg_return_pct=0, avg_trade_bars=0, avg_profit=0, avg_profit_pct=0, avg_winning_trade_bars=0, avg_loss=0, avg_loss_pct=0, avg_losing_trade_bars=0, largest_win=0, largest_win_pct=0, largest_win_bars=0, largest_loss=0, largest_loss_pct=0, largest_loss_bars=0, max_wins=0, max_losses=0, sharpe=0, sortino=0, calmar=None, profit_factor=0, ulcer_index=0, upi=0, equity_r2=0, std_error=0, annual_std_error=

## İleriye Yönelik Analiz


**PyBroker**, geriye dönük test gerçekleştirmek için [İleriye Doğru Analiz](https://www.youtube.com/watch?v=WBZ_Vv-iMv4) olarak bilinen güçlü bir algoritma kullanır. Algoritma, arka test verilerini, her biri verinin tren-test bölümünü içeren sabit sayıda zaman penceresine böler.

İleriye Doğru Analiz algoritması daha sonra, gerçek dünyada bir ticaret stratejisinin yürütüleceği şekilde, zamanda "ileriye doğru yürümeye" devam eder. Model ilk önce en erken pencerede eğitilir ve daha sonra bu penceredeki test verileri üzerinden değerlendirilir.

Algoritma zaman içinde bir sonraki pencereyi değerlendirmek için ilerledikçe, önceki penceredeki test verileri eğitim verilerine eklenir. Bu süreç tüm zaman pencereleri değerlendirilinceye kadar devam eder.

![İleriye Doğru Şeması](https://github.com/edtechre/pybroker/blob/master/docs/_static/walkforward.png?raw=true)

Walkforward Analiz algoritması, bu yaklaşımı kullanarak bir ticaret stratejisinin gerçek dünyadaki performansını simüle edebilir ve daha güvenilir ve doğru geriye dönük test sonuçları üretebilir.

Daha önce eğittiğimiz [LinearRegression](https://scikit-learn.org/stable/modules/generate/sklearn.linear_model.LinearRegression.html) modelinden alım ve satım sinyalleri üreten bir ticaret stratejisini düşünelim. Strateji ``hold_long``` işlevi olarak uygulanır:

In [9]:
def hold_long(ctx):
    if not ctx.long_pos():
        # Bir sonraki çubuğun olumlu bir getiri sağlayacağı öngörülüyorsa satın alın:
        if ctx.preds('slr')[-1] > 0:
            ctx.buy_shares = 100
    else:
        # Bir sonraki çubuğun negatif getiri sağlayacağı tahmin ediliyorsa satış yapın:
        if ctx.preds('slr')[-1] < 0:
            ctx.sell_shares = 100
            
strategy.clear_executions()
strategy.add_execution(hold_long, ['FROTO.IS', 'EREGL.IS'], models=model_slr)

``hold_long``` işlevi, model bir sonraki çubuk için pozitif bir getiri tahmin ettiğinde uzun bir pozisyon açar ve ardından model negatif bir getiri tahmin ettiğinde konumu kapatır.

Yapılan tahminlere erişmek için [ctx.preds('slr')](https://www.pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext.preds) yöntemi kullanılır işlevde yürütülen mevcut hisse senedi sembolü için ```'slr'``` modeliyle (NVDA veya AMD). Tahminler bir [NumPy dizisinde](https://numpy.org/doc/stable/reference/generate/numpy.array.html) saklanır ve mevcut hisse senedi sembolü için en son tahmine ``` kullanılarak erişilir. ctx.preds('slr')[-1]```, bu, modelin bir sonraki çubuğun dönüşüne ilişkin tahminidir.

Artık bir ticaret stratejisi tanımladığımıza ve ```'slr'``` modelini kaydettiğimize göre, Walkforward Analiz algoritmasını kullanarak geriye dönük testi çalıştırabiliriz.

Geri test, ``Strategy`` üzerinde [walkforward](https://www.pybroker.com/en/latest/reference/pybroker.strategy.html#pybroker.strategy.Strategy.walkforward) yöntemi çağrılarak çalıştırılır. İstenilen sayıda zaman penceresi ve eğitim/test bölme oranı ile nesne. Bu durumda, her biri 50/50 tren-test ayrımına sahip 3 zaman penceresi kullanacağız.

Ayrıca ```'slr'``` modelimiz gelecekte bir çubuk için tahmin yaptığı için ``lookahead``` parametresini ```1``` olarak belirtmemiz gerekiyor. Bu, eğitim verilerinin test sınırına sızmamasını sağlamak için gereklidir. ``lookahead``` parametresi her zaman gelecekte tahmin edilen çubuk sayısına ayarlanmalıdır.

In [10]:
result = strategy.walkforward(
    warmup=20, 
    windows=3, 
    train_size=0.5, 
    lookahead=1, 
    calc_bootstrap=True
)

Backtesting: 2017-03-01 00:00:00 to 2022-03-01 00:00:00

Loading bar data...
[*********************100%***********************]  1 of 1 completed
Loaded bar data: 0:00:00 

Computing indicators...


  0% (0 of 1) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--
100% (1 of 1) |##########################| Elapsed Time: 0:00:00 ETA:  00:00:00
100% (1 of 1) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00



Train split: 2017-03-08 00:00:00 to 2018-05-28 00:00:00
EREGL.IS R^2=-0.05324926162898347
FROTO.IS R^2=-0.0047909961822389935
Finished training models: 0:00:00 

Test split: 2018-05-29 00:00:00 to 2019-08-16 00:00:00


  0% (0 of 319) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--
 50% (161 of 319) |###########           | Elapsed Time: 0:00:00 ETA:  00:00:00
100% (319 of 319) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Train split: 2018-05-29 00:00:00 to 2019-08-16 00:00:00
EREGL.IS R^2=-0.05372138271192117
FROTO.IS R^2=-0.059907976342019165
Finished training models: 0:00:00 

Test split: 2019-08-19 00:00:00 to 2020-11-20 00:00:00


  0% (0 of 319) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--
100% (319 of 319) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Train split: 2019-08-19 00:00:00 to 2020-11-20 00:00:00
EREGL.IS R^2=-0.014160870768241685
FROTO.IS R^2=-0.04725413090756847
Finished training models: 0:00:00 

Test split: 2020-11-23 00:00:00 to 2022-02-28 00:00:00


  0% (0 of 319) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--
 59% (191 of 319) |#############         | Elapsed Time: 0:00:00 ETA:  00:00:00
100% (319 of 319) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Calculating bootstrap metrics: sample_size=100, samples=10000...
Calculated bootstrap metrics: 0:00:04 

Finished backtest: 0:00:07


Walkforward Analiz algoritmasını kullanan geriye dönük test işlemi sırasında, ```'slr'``` modeli belirli bir pencerenin eğitim verileri üzerinde eğitilir ve ardından ```hold_long``` işlevi aynı pencerenin test verileri üzerinde çalıştırılır.

Model, bir sonraki günün getirileri hakkında tahminlerde bulunmak için eğitim verileriyle eğitilir. Daha sonra ```hold_long``` işlevi bu tahminleri geçerli günün işlem seansına yönelik alım veya satım kararları vermek için kullanır.

Her bir pencere için test verileri üzerinde ticaret stratejisinin performansını değerlendirerek, stratejinin gerçek dünya ticaret koşullarında ne kadar iyi performans göstereceğini görebiliriz. Bu süreç, ticaret stratejisinin genel performansını değerlendirmek için sonuçları kullanarak, arka testteki her zaman penceresi için tekrarlanır:

In [11]:
result.metrics_df

,name,value
0,trade_count,67.000000
1,initial_market_value,100000.000000
2,end_market_value,109423.000000
3,total_pnl,9008.000000
4,unrealized_pnl,415.000000
5,total_return_pct,9.008000
6,total_profit,20686.000000
7,total_loss,-11678.000000
8,total_fees,0.000000
9,max_drawdown,-9131.000000


In [12]:
result.bootstrap.conf_intervals

lower     upper
name          conf                     
Profit Factor 97.5%  0.389781  2.464466
              95%    0.465539  2.150435
              90%    0.560400  1.843387
Sharpe Ratio  97.5% -0.176622  0.217270
              95%   -0.147028  0.191185
              90%   -0.110889  0.156415

In [13]:
result.bootstrap.drawdown_conf

,amount,percent
conf,,
99.9%,-12458.0,-10.791081
99%,-9836.0,-8.542115
95%,-7305.0,-6.497609
90%,-6254.0,-5.600556


Özetle, Walkforward Analizi'nin yardımıyla **PyBroker** kullanarak doğrusal bir regresyon modelini eğitme ve geriye dönük test etme sürecini tamamladık. Gördüğümüz ölçümler, arka testteki tüm zaman pencerelerinden elde edilen test verilerine dayanmaktadır. Ticaret stratejimizin iyileştirilmesi gerekmesine rağmen **PyBroker**'da bir modelin nasıl eğitilip değerlendirileceği konusunda iyi bir anlayışa sahip olduk.

Regresyon analizi yapmadan önce, [homoscedasticity](https://en.wikipedia.org/wiki/Homoscedasticity_and_heteroscedasticity), artıkların normalliği vb. gibi belirli varsayımları doğrulamanın önemli olduğunu lütfen unutmayın. Ayrıntıları vermedim. Kısalık olması açısından bu varsayımları burada paylaşıyorum ve bu alıştırmayı kendi başınıza yapmanızı tavsiye ediyorum.

**PyBroker**'da yalnızca doğrusal regresyon modelleri oluşturmakla da sınırlı değiliz. Gradyan destekli makineler, sinir ağları veya seçtiğimiz diğer mimariler gibi diğer model türlerini eğitebiliriz. Bu esneklik, belirli ticaret hedeflerimiz için en iyi performansı gösteren modeli bulmak amacıyla çeşitli modelleri ve yaklaşımları keşfetmemize ve denememize olanak tanır.

PyBroker ayrıca, duruma göre modelimiz için bir [input_data_fn](https://www.pybroker.com/en/latest/reference/pybroker.model.html#pybroker.model.model) belirtme yeteneği gibi özelleştirme seçenekleri de sunar. giriş verilerinin nasıl oluşturulduğunu özelleştirmemiz gerekiyor. Tahmin yapmak için birden fazla geçmiş değer kullanan otoregresif modeller (yani ARMA veya RNN) için girdi oluştururken bu gerekli olacaktır. Benzer şekilde, tahminlerin nasıl yapıldığını özelleştirmek için kendi [predict_fn](https://www.pybroker.com/en/latest/reference/pybroker.model.html#pybroker.model.model) modelimizi belirtebiliriz (varsayılan olarak, modelin ```tahmin``` fonksiyonu çağrılır).

Bu bilgiyle **PyBroker**'da kendi modellerinizi ve ticaret stratejilerinizi oluşturmaya ve test etmeye başlayabilir ve bu çerçevenin sunduğu geniş olanakları keşfetmeye başlayabilirsiniz!